In [1]:
import torch
import json
import os
import warnings

warnings.simplefilter("ignore")

In [9]:
def load_data_setting(setting):
    folder = "../exps"
    json_file = "data_config.json"
    json_path = os.path.join(folder, setting, json_file)
    with open(json_path, "r") as f:
        config = json.load(f)

    permutation_seed = config["permutation_seed"]
    test_size = config["test_size"]
    partition_seed = config["partition_seed"]
    n_class = config["n_class"]
    
    data_tensors = config["data_tensors"]
    loaded_data_tensors = torch.load(data_tensors)
    data_tr = loaded_data_tensors["data_tr"]
    label_tr = loaded_data_tensors["label_tr"]
    data_te = loaded_data_tensors["data_te"]
    label_te = loaded_data_tensors["label_te"]

    return permutation_seed, test_size, partition_seed, n_class, data_tr, label_tr, data_te, label_te

In [10]:
permutation_seed, test_size, partition_seed, n_class, data_tr, label_tr, data_te, label_te = load_data_setting("Setting_1")

In [11]:
n_class

10